```{image} ../images/logos/Xarray_Logo_RGB_Final.png
:width: 250px
:align: center
```

# Introducción a Xarray
---

`Xarray` es una librería de Python diseñada para facilitar el análisis y manipulación de **datos multidimensionales etiquetados** (como los que provienen de modelos climáticos, observaciones satelitales o simulaciones numéricas).

Inspirado en la estructura de `Pandas`, `Xarray` extiende su funcionalidad a arreglos N-dimensionales, permitiendo trabajar de forma intuitiva con datos que tienen múltiples coordenadas como **tiempo**, **latitud** y **longitud**.

> Xarray es especialmente popular en ciencias de la Tierra y del clima, donde los datos suelen venir en formatos como **NetCDF**, **Zarr** o **GRIB**.


## 📚 ¿Qué vas a aprender?

Al completar este cuadernillo serás capaz de:

- Comprender qué son `DataArray` y `Dataset`
- Crear objetos Xarray desde arreglos de NumPy
- Asignar dimensiones y coordenadas
- Explorar y seleccionar datos usando coordenadas
- Calcular estadísticas y aplicar operaciones
- Visualizar datos con `.plot()`
- Leer datos reales desde archivos NetCDF


## ✅ Requisitos previos

| Concepto | Importancia | Notas |
| --- | --- | --- |
| [Introducción a NumPy](./1.2Numpy-Pandas-Xarray.ipynb) | Necesario | Arreglos multidimensionales |
| [Fundamentos de Pandas](./1.2Numpy-Pandas-Xarray.ipynb) | Necesario | Indexado y manipulación tabular |
| [Uso de JupyterLab](./1.1Fundamentos-python.ipynb) | Útil | Navegación entre celdas |

⏱️ **Tiempo estimado de aprendizaje**: 20–25 minutos  
✍️ **Formato**: interactivo, ejecuta y modifica el código a medida que avanzas

## 1. Primeros pasos con Xarray 🧪
---

La unidad básica en Xarray es el **`DataArray`**, que extiende la funcionalidad de los `ndarray` de NumPy. A diferencia de NumPy, Xarray permite asignar:

- **Dimensiones con nombre** (`time`, `lat`, `lon`, etc.)
- **Coordenadas** con etiquetas asociadas
- **Atributos** descriptivos como unidades, nombre estándar, etc.

Esta herramienta es particularmente poderosa cuando se trabaja con datos atmosféricos, oceanográficos o climáticos provenientes de modelos numéricos o sensores remotos, donde las variables dependen del tiempo, la latitud, la longitud y a veces también de la altitud.

Importemos las librerias necesarias para este cuadernillo

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd

### 🎛️ 1.1. Crear un `DataArray` desde NumPy

Para comenzar, creamos una matriz tridimensional con valores simulados de temperatura (en grados Kelvin). Esta matriz tendrá dimensiones que representan el tiempo, la latitud y la longitud:


In [ ]:
data = 283 + 5 * np.random.randn(5, 3, 4)  # 5 tiempos, 3 latitudes, 4 longitudes
data.shape

Podemos construir un `DataArray` inicial directamente desde este arreglo de NumPy, aunque sin información adicional aún:

In [ ]:
temp = xr.DataArray(data)
temp

⚠️ Como no definimos nombres para las dimensiones, Xarray usa nombres genéricos como `dim_0`, `dim_1`, `dim_2`, lo cual puede dificultar la interpretación de los datos.

### 1.2 Asignar nombres de dimensiones 📀

Para mejorar la legibilidad, asignamos nombres explícitos a las dimensiones. Esto convierte un arreglo anónimo en un conjunto de datos más descriptivo y navegable:

In [ ]:
temp = xr.DataArray(data, dims=["time", "lat", "lon"])
temp

Creamos un objeto con datos y dimensiones que gráficamente se podría representar de la siguiente manera:

<img src="../images/datarray.png" width=600 alt="Datarray"></img> 

Esto nos permite realizar operaciones sobre dimensiones por nombre, en lugar de índices numéricos.

### 1.3 Agregar coordenadas a las dimensiones 🗺️

Además de los nombres, podemos asociar coordenadas reales que representen valores geográficos o temporales. Esto permite, por ejemplo, ubicar espacialmente los datos o hacer filtrados temporales:

In [ ]:
times = pd.date_range("2018-01-01", periods=5)
lats = np.linspace(25, 55, 3)
lons = np.linspace(-120, -90, 4)

Construimos nuevamente el `DataArray`, esta vez incluyendo coordenadas:

In [ ]:
temp = xr.DataArray(data, coords=[times, lats, lons], dims=["time", "lat", "lon"])
temp

Con esto obtenemos un objeto con visualización enriquecida: las dimensiones están etiquetadas y cada punto tiene una referencia espacial y temporal clara.

<img src="../images/datarray_coords.png" width=600 alt="Datarray_Coords"></img> 

### 1.4 Agregar atributos descriptivos 🏷️

Finalmente, Xarray permite incluir metadatos como unidades físicas, nombres estándar de variables o descripciones adicionales. Esto es importante para la trazabilidad y para facilitar el trabajo colaborativo o la publicación de datos:


In [ ]:
temp.attrs["units"] = "K"
temp.attrs["standard_name"] = "air_temperature"

Estos atributos no alteran los datos, pero se usan al exportar, graficar o convertir los objetos a otros formatos como netCDF. También ayudan a los usuarios a entender rápidamente qué representa el conjunto de datos.


## 2. Dataset: Contenedor multidimensional 🧳
---
Mientras que un `DataArray` representa una sola variable con sus dimensiones y coordenadas, un `Dataset` es un **contenedor que puede almacenar múltiples `DataArray`s**, que posiblemente compartan dimensiones y coordenadas.

Esto es muy útil en ciencias ambientales, donde queremos trabajar simultáneamente con variables como temperatura, presión, humedad, etc., dentro de un mismo archivo o conjunto de datos.


### 2.1 Crear múltiples DataArrays 🌡️

Partimos de los `DataArray`s que representan distintas variables físicas como temperatura del aire, presión atmosférica y humedad relativa:


In [ ]:
# Temperatura
temp = xr.DataArray(
    data=283 + 5 * np.random.randn(5, 3, 4),
    coords={"time": times, "lat": lats, "lon": lons},
    dims=["time", "lat", "lon"],
    name="Temperature",
    attrs={"units": "K", "standard_name": "air_temperature"}
)

# Presión
pressure = xr.DataArray(
    data=1000 + 10 * np.random.randn(5, 3, 4),
    coords={"time": times, "lat": lats, "lon": lons},
    dims=["time", "lat", "lon"],
    name="Pressure",
    attrs={"units": "hPa", "standard_name": "air_pressure"}
)

# Humedad relativa
hr = xr.DataArray(
    data=np.random.uniform(60, 100, size=5),
    coords={"time": times},
    dims=["time"],
    name="RelativeHumidity",
    attrs={"units": "%", "standard_name": "relative_humidity"}
)

### 2.2 Crear el Dataset 📦

Una vez definidos los `DataArray`s, podemos crear un `Dataset` pasando un diccionario con las variables:

In [ ]:
ds = xr.Dataset(
    data_vars={
        "Temperature": temp,
        "Pressure": pressure,
        "RelativeHumidity": hr
    }
)

En un entorno interactivo como JupyterLab, este `Dataset` se mostrará como una menu desplegable que incluye:
- Dimensiones (`dims`)
- Coordenadas (`coords`)
- Variables (`data_vars`)
- índices (`indexes`)
- Atributos (`attrs`)

In [ ]:
ds

La visualización del `Dataset` mostrará las dimensiones compartidas, las coordenadas y un resumen de cada variable contenida como se muestra en la siguiente imagen tomada de la documentación de `xarray`

<img src="../images/xarray-datastructure.png" width=800 alt="Dataset"></img> 

Esto permite acceder rápidamente a las dimensiones comunes y explorar todas las variables de forma estructurada.


## 3. Selección de datos y operaciones básicas 🧩
---
Una vez que tenemos nuestros datos organizados en `DataArray` o `Dataset`, Xarray nos permite acceder, filtrar y operar sobre ellos de forma muy intuitiva utilizando etiquetas en lugar de índices numéricos.


### 3.1 Acceder a variables en un Dataset 🔎

Cada variable contenida dentro de un `Dataset` es un `DataArray`, y se puede acceder de manera sencilla usando notacion por atributo `ds.<atributo>` (acceso directo si el nombre es válido) 

In [ ]:
ds.Temperature

O Notación por clave de diccionario (más general y flexible)

In [ ]:
ds["Pressure"]

Ambas formas retornan el `DataArray` correspondiente. La primera es más compacta, útil en notebooks interactivos; la segunda es más robusta, especialmente si los nombres contienen espacios, guiones u otros caracteres especiales.


### 3.2 Seleccionar datos por coordenadas con `.sel()` 🧭

El método `.sel()` permite seleccionar subconjuntos de datos utilizando los **valores de las coordenadas** en lugar de los índices posicionales. Esto hace que el código sea más legible y expresivo.


Por ejemplo, podemos seleccionar todos los datos para una fecha específica


In [ ]:
ds.sel(time="2018-01-01")

O seleccionar una variable y una ubicación específica

In [ ]:
ds.Temperature.sel(time="2018-01-01", lat=25, lon=-120)


✅ Muy útil y conveniente cuando trabajamos con fechas o ubicaciones reales.

### 3.3 Seleccionar datos por posición con `.isel()` 🔹

Si en cambio queremos seleccionar por posición (como con NumPy), usamos `.isel()`. Esto es útil cuando no conocemos las coordenadas exactas o estamos en procesos de automatización.


In [ ]:
# Seleccionar el primer tiempo (posición 0)
ds.isel(time=0)

Seleccionar temperatura para la primera posición de latitud y segunda longitud a lo largo del tiempo

In [ ]:
ds.Temperature.isel(lat=0, lon=1)

🧠 `.isel()` usa números de posición; `.sel()` usa etiquetas de coordenadas.

### 3.4 Seleccionar rangos con `slice()` 🪄

Podemos seleccionar rangos utilizando `slice()`, lo que facilita el trabajo con subconjuntos:


Por ejemplo seleccionemos el subconjunto de datos entre dos fechas

In [ ]:
ds.sel(time=slice("2018-01-01", "2018-01-03"))

O subconjunto de latitudes

In [ ]:
ds.sel(lat=slice(30, 50))

Esto es especialmente útil en análisis temporales o espaciales que requieren ventanas móviles.

## 4. Operaciones estadísticas y gráficas 📊
---
Una de las grandes ventajas de Xarray es que permite aplicar operaciones estadísticas y crear visualizaciones de forma muy similar a Pandas, pero extendidas a múltiples dimensiones.

### 4.1 Estadísticas sobre dimensiones 📐

Podemos calcular estadísticas como la media, desviación estándar, máximo o mínimo a lo largo de una o varias dimensiones.


In [ ]:
# Media de temperatura a lo largo del tiempo
ds.Temperature.mean(dim="time")

In [ ]:
# Mínimo de presión sobre latitudes
ds.Pressure.min(dim="lat")

In [ ]:
# Desviación estándar de humedad relativa
ds.RelativeHumidity.std(dim="time")

✅ Estas funciones son muy útiles para obtener resúmenes climatológicos o patrones promedios.


4.2 Interpolación de datos 🌐
Cuando necesitas obtener valores en coordenadas que no existen exactamente en tus datos, puedes usar `.interp()` para interpolar:

In [ ]:
# Interpolación espacial
valor_interp = ds.Temperature.interp(lat=33.5, lon=-110)
valor_interp

📌 Puedes especificar el método de interpolación (como linear, nearest) y ajustar la tolerancia.

### 4.3 Visualizaciones rápidas con `.plot()` 🖼️

Xarray incluye integración con Matplotlib para hacer gráficos rápidos desde cualquier `DataArray`:

Generemos una serie temporal de temperatura en una ubicación determinada

In [ ]:
serie = ds.Temperature.sel(lat=40, lon=-100, method="nearest")
serie.plot()

O un mapa de temperatura para t=0

In [ ]:
mapa = ds.Temperature.isel(time=0)
mapa.plot()

O un Mapa de temperatura promedio

In [ ]:
prom_mapa = ds.Temperature.mean("time")
prom_mapa.plot()

🔍 Estas funciones detectan automáticamente si el resultado es 1D o 2D y generan una línea o un mapa respectivamente.


### 4.4 Aplicar máscaras y condiciones 🎭

Puedes aplicar filtros condicionales a los datos usando `.where()`, muy útil para enmascarar regiones o aplicar umbrales:


In [ ]:
# Mostrar datos para latitudes menores a 50 grados
ds.Temperature.where(ds.lat < 50).isel(time=0).plot()

In [ ]:
# Filtrar valores de presión mayores a cierto umbral
ds.Pressure.where(ds.Pressure > 1000).isel(time=1).plot()

🧪 `.where()` mantiene la forma del arreglo pero oculta los datos que no cumplen la condición.

## 5. 📂 Lectura de archivos netCDF con Xarray
---
Esta sección te introduciría a cómo cargar datos reales desde archivos científicos en formato netCDF, que es uno de los más comunes en ciencias de la atmósfera, océano y clima.

Los archivos NetCDF (Network Common Data Form) son ampliamente utilizados para almacenar datos multidimensionales del clima, el océano y la atmósfera. Xarray proporciona una forma sencilla y poderosa para leer y explorar este tipo de archivos.


### 5.1 Leer un archivo netCDF

Xarray facilita la lectura de estos archivos con la función `xr.open_dataset()`.

En este ejemplo vamos a usar un archivo real llamado `sst.mnmean.nc`, que contiene datos mensuales de la **temperatura superficial del mar (SST)** a nivel global durante varias décadas. Este tipo de dataset es comúnmente utilizado para analizar fenómenos como El Niño, anomalías de temperatura, o climatologías oceánicas.


In [ ]:
path_ncfile = "../data/sst.mnmean.nc"
ds = xr.open_dataset(path_ncfile)

In [ ]:
ds

### 5.2 Explorar estructura del archivo 🔎

Una vez abierto el Dataset, puedes inspeccionar las variables y dimensiones disponibles:
- Variables contenidas (por ejemplo, `sst` para temperatura superficial del mar)
- Dimensiones como `time`, `lat`, y `lon`
- Coordenadas disponibles y sus unidades
- Atributos globales (fuente, descripción, institución, etc.)

In [ ]:
# Visualizar las dimensiones y coordenadas
print(ds.dims)
print(ds.coords)

También puedes ver los atributos generales del archivo:

In [ ]:
ds.attrs

### 5.3 Acceder a variables específicas 🧬

Cada variable contenida en el archivo se puede acceder directamente como un `DataArray`.


In [ ]:
# Seleccionar la variable de temperatura superficial del mar (sst)
sst = ds['sst']
sst

Esto mostrará la forma, dimensiones y atributos asociados a la variable.

### 5.4 Visualización básica 🌍

Vamos a generar un mapa de la temperatura para el primer valor temporal del archivo.


In [ ]:
# Graficar la primera capa temporal
sst.isel(time=0).plot(cmap="coolwarm")

🧭 Esto te muestra un mapa global de temperatura superficial del mar para el primer mes disponible en el conjunto de datos.


✅ Puedes cambiar el índice o usar `.sel(time=...)` para seleccionar fechas específicas si el tiempo está en formato calendario.


### 5.5 Operaciones comunes ⚙️

Ya que tenemos un `DataArray`, podemos aplicar operaciones estadísticas o espaciales directamente:


In [ ]:
# Temperatura promedio global (promedio espacial)
sst.mean(dim=["lon", "lat"]).plot()

### 5.6 Aplicar máscaras o filtros 🎭

Filtrar solo los valores en el hemisferio sur o para latitudes específicas:


In [ ]:
sst.where(sst.lat < 0).isel(time=0).plot(cmap="viridis")

### 5.7 Guardar resultados en nuevo archivo 💾

Podemos guardar los datos procesados a un nuevo archivo Zarr:


In [ ]:
sst_mean = sst.mean(dim="time")
sst_mean.to_zarr("sst_media.zarr")

In [ ]:
! ls sst_media.zarr

🎉 ¡Con esto completamos nuestra introducción al uso de Xarray con datos reales en formato netCDF!

---

## ✅ Conclusiones

En este cuadernillo aprendiste los fundamentos del manejo de datos multidimensionales con **Xarray**, una herramienta esencial en la ciencia del clima, meteorología y disciplinas relacionadas con datos ambientales.

🔑 **Lo más destacado**:

- Comprendiste la diferencia entre `DataArray` y `Dataset`, dos estructuras clave para manejar datos con múltiples dimensiones y coordenadas.
- Aprendiste a **etiquetar** tus datos con dimensiones y coordenadas que hacen tu análisis más claro y reproducible.
- Aplicaste operaciones estadísticas como `mean()`, `std()` o `sum()` a lo largo de diferentes dimensiones.
- Usaste `.sel()` y `.isel()` para extraer datos fácilmente con etiquetas o índices.
- Generaste visualizaciones rápidas usando `.plot()` para explorar tendencias temporales y mapas espaciales.
- Accediste y exploraste datos reales en formato `netCDF`, el estándar para datos climáticos y atmosféricos.

📦 Con este conocimiento ya puedes comenzar a trabajar con grandes volúmenes de datos científicos como modelos climáticos, observaciones satelitales o reanálisis atmosféricos.

---

## 📚 Recursos recomendados

- [Documentación oficial de Xarray](https://docs.xarray.dev/)
- [Pythia Foundations: Introducción a Xarray](https://foundations.projectpythia.org/core/xarray.html)
- [Earth & Environmental Data Science](https://earth-env-data-science.github.io/intro.html)

---

🎓 ¡Felicitaciones! Estás listo para abordar análisis más complejos con datos multidimensionales usando Xarray.



## Fuentes y Referencias

* Rose, B. E. J., Kent, J., Tyle, K., Clyne, J., Banihirwe, A., Camron, D., May, R., Grover, M., Ford, R. R., Paul, K., Morley, J., Eroglu, O., Kailyn, L., & Zacharias, A. (2023). Pythia Foundations (Version v2023.05.01) https://doi.org/10.5281/zenodo.7884572